# 追踪功能`Tracing`
强大的工具，辅助深入理解、调试和监控Agent的运行过程

## Motivation
为什么日志`logging`还不够，还得用追踪？
- 日志为离散数据，记录时间点与事件
多路由、步骤、Agent、工具的Workflow需要理解：
- 整个流程的全貌：input to output，整个过程经历了什么阶段
- 事件之间的关联性：“Agent调用Tools”，“该工具调用是哪个父任务的一部分”
- 时间关系和性能的瓶颈：“某个步骤耗时过长”，“哪些步骤是并行执行的”

`Tracing`就是为了解决这些问题而设计，不仅仅记录单个时间，而是记录事件之间的关系和时间跨度。
将一次完整的端到端操作（比如用户问一个问题，Agent调用工具，最终给出答案）表示为一个Trace (追踪记录)，并将这个Trace分解为多个Span (跨度)，每个Span代表其中的一个操作单元（比如一次LLM调用、一次工具调用）。

通过追踪可以进行整个完整事件执行的子事件切片，进行诊断、理解、观察、优化

## Definition
### Trace（追踪记录）
代表一个完整的工作流或请求的端到端记录。由一系列的Span组成。
核心属性：
- `trace_id`：唯一标识，用于追踪整个请求的生命周期
- `workflow_name`：工作流名称（如“翻译任务”）
- `group_id`(可选)：用于将多次相关的Trace关联起来（如同相关的工作流事件的关联对话）
- `metadata`（可选）：可以附加一些自定义的元数据，比如用户ID、任务类型、请求来源等

### Span（跨度）
代表Trace中的一个具体的操作单元，有明确的开始和结束时间。
Span可以嵌套，形成父子关系。
- `start_time`：记录Span开始的时间
- `end_time`：记录Span结束的时间
- `trace_id`：它所归属的Trace的ID
- `span_id`：唯一标识，用于追踪单个操作单元
- `parent_span_id`/`parent_id`：可选，用于表示当前Span的父Span，形成Span树结构
- `span_data`:包含该操作的具体细节，SDK根据操作类型提供了不同的`SpanData`子类，如：
    - `AgentSpanData`:记录哪个Agent运行了，以及它的配置参数
    - `GenerationSpanData`:记录LLM调用的输入、输出、模型名称等
    - `FunctionSpanData`:记录工具（函数）调用的输入、输出
    - `GuardrailSpanData`,`HandoffSpanData`,`TranscriptionSpanData`等

## Design & Implementation
`openai-agent`SDK中的追踪
### 默认追踪行为
`openai-agent`SDK默认开启追踪功能，所有的操作都会创建Trace和Span：
- 每次调用`Runner.run()`、`run_sync()`、`run_streamed()`等运行方法时，都会自动创建一个根Trace（默认名为`Agent trace`）
- 拆解此`Trace`内部：
    - 每个Agent的运行都会被包裹在`agent_span()`中
    - 每次LLM调用会被包裹在`llm_span()`中
    - 每次函数工具调用会被包裹在`function_span()`中
    - 护栏触发（`guardrail_span()`）、任务交接（`handoff_span()`）、语言处理（`transcription_span()`）、`speech_span()`等操作也会被包裹在对应的Span中

这意味着不需要手动添加追踪代码

>此处的SDK指的是什么？

### 禁用追踪
如果你不需要追踪功能（比如性能要求极其高的生产环境中，或者只是想让输出更干净），可以通过一以下都是禁用：
- 环境变量（全局禁用）：`export OPENAI_AGENTS_DISABLE_TRACING=1`
- 代码中设置（局部禁用，如单个文件）：设置`set_tracing_disabled(disable=True)`
- RunConfig(单次运行禁用)：`Runner.run(...,run_config=RunConfig(tracing_disabled=True))`

### 高阶追踪：将多次运行合并到一个Trace
默认情况下，每次`Runner.run()`调用会创建一个新的Trace。
但是需要进行逻辑上相关的多次运行（比如一个多步骤工作流中的不同阶段）归属到同一个Trace下，一边查看完整端到端的执行过程。

示例`trace()`上下文管理器：
```python
from agents import Agent, Runner, trace
import asyncio
# ... (LLM配置, Agent定义等) ...

async def main():
    agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

    # 使用 trace() 将两次 Runner.run 调用包裹起来
    with trace("Joke workflow"): # "Joke workflow" 是这次合并追踪的名称
        # 第一次运行：让Agent讲个笑话
        first_result = await Runner.run(agent, "Tell me a joke")
        
        # 第二次运行：让Agent评价上一个笑话
        # 这次运行产生的Spans会自动成为 "Joke workflow" Trace 的一部分
        second_result = await Runner.run(agent, f"Rate this joke: {first_result.final_output}")
        
        print(f"Joke: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# ... (运行代码) ...
```

### 自动化追踪处理器（`set_trace_processors()`）
SDK默认的追踪处理器可能只是将追踪数据打印到控制台或存储在内存中。如果你想将这些宝贵的追踪数据发送到专门的可视化平台（如 LangSmith 或 Langfuse）进行分析和长期存储，就需要替换默认的处理器。

>重新实操一遍
1. LangSmith集成
- 安装依赖：`pip install langsmith`
- 配置环境变量：`export LANGSMITH_API_KEY=your_api_key`
- 设置追踪处理器：`set_trace_processors([LangSmithTraceProcessor()])`
```python
import asyncio
from agents import Agent, Runner, set_trace_processors
from langsmith.wrappers import OpenAIAgentsTracingProcessor
from agents.extensions.models.litellm_model import LitellmModel
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

async def main():
    agent = Agent(
        name="Captain Obvious",
        instructions="You are Captain Obvious, the world's most literal technical support agent.",
        model=llm,
    )

    question = "Why is my code failing when I try to divide by zero? I keep getting this error message."
    result = await Runner.run(agent, question)
    print(result.final_output)

if __name__ == "__main__":
    set_trace_processors([OpenAIAgentsTracingProcessor()])
    asyncio.run(main())
```

2. Langfuse集成
```python

```